In [1]:
import pandas as pd
from annoy import AnnoyIndex
import json
import os

In [ ]:
EMB_PATH = "../data/embeddings_clip.csv"
INDEX_DIR = "../api/indexes"
os.makedirs(INDEX_DIR, exist_ok=True)


In [3]:
# --- Load embeddings ---
df = pd.read_csv(EMB_PATH)
df["embedding"] = df["embedding"].apply(lambda x: list(map(float, x.split(","))))

emb_dim = len(df["embedding"].iloc[0])
index = AnnoyIndex(emb_dim, "angular")

print(f"Embedding dim = {emb_dim}")

Embedding dim = 512


In [5]:
print("Building Annoy index...")

for i, row in df.iterrows():
    index.add_item(i, row["embedding"])

# More trees → better precision, slower build
index.build(50)
index.save(f"{INDEX_DIR}/fashion_index.ann")

print("Saving metadata...")
meta = df.drop(columns=["embedding"]).to_dict(orient="records")

with open(f"{INDEX_DIR}/metadata.json", "w") as f:
    json.dump(meta, f)

print("Done. Index saved at", INDEX_DIR)

Building Annoy index...
Saving metadata...
Done. Index saved at api/indexes
